In [108]:
import pandas as pd
import numpy as np

#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

deliveries = pd.read_csv("Data/deliveries.csv")
match = pd.read_csv("Data/matches.csv")
matches = match[["city","venue","season","toss_winner","toss_decision","team1","team2","winner"]]
matches.head()

,city,venue,season,toss_winner,toss_decision,team1,team2,winner
0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",2017,Royal Challengers Bangalore,field,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad
1,Pune,Maharashtra Cricket Association Stadium,2017,Rising Pune Supergiant,field,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant
2,Rajkot,Saurashtra Cricket Association Stadium,2017,Kolkata Knight Riders,field,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders
3,Indore,Holkar Cricket Stadium,2017,Kings XI Punjab,field,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab
4,Bangalore,M Chinnaswamy Stadium,2017,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore


In [109]:
def factorize_fields(matches):
    df = matches[["team1","team2","toss_winner","winner"]]
    _, b = pd.factorize(df.values.T.reshape(-1, ))  
    facorized_fields = df.apply(lambda x: pd.Categorical(x, b).codes)
    matches["venue_"] = matches.venue.factorize()[0]
    matches["city_"] = matches.city.factorize()[0]
    matches["toss_winner_"] = matches.toss_winner.factorize()[0]
    matches["toss_decision_"] = matches.toss_decision.factorize()[0]
    matches = pd.concat([matches[["city_","venue_","season","toss_winner_","toss_decision_"]], facorized_fields], 1)
    factors = pd.factorize(df.values.T.reshape(-1, ))[1]
    return [factors, matches]

In [110]:
[factors, matches] = factorize_fields(matches)

/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [111]:
# Removing season: Include if time model made
matches.drop(["season"],1,inplace=True)

In [112]:
def remap(item):
    return factors[item]

In [113]:
def model_building(model, predictors, outcome, data, test_data):
    model.fit(data[predictors], data[outcome])  
    kf = KFold(data.shape[0], n_folds=5)
    error = []
    for train, test in kf:
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    cv_error = np.mean(error)
    #print('Cross validation Score : %s' % '{0:.3%}'.format(cv_error))
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(test_data[predictors])
    test_data["predicted_winner"] = predictions
    accuracy = metrics.accuracy_score(predictions,test_data[outcome])
    #print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    test_data["Team1"] = test_data["team1"].apply(remap)
    test_data["Team2"] = test_data["team2"].apply(remap)
    test_data["Actual Winner"] = test_data["winner"].apply(remap)
    test_data["Predicted Winner"] = test_data["predicted_winner"].apply(remap)
    df = test_data[["Team1","Team2","Actual Winner", "Predicted Winner"]]
    return [df, accuracy, cv_error]

In [114]:
# Temporary train-test split: to be removed after obtaining the test data
ind = int(0.2*len(matches))
test_data = matches[:ind]
train_data = matches[ind:]

In [115]:
model = RandomForestClassifier(n_estimators=100)
output = ['winner']
predictors = matches.drop(["winner"],1).columns
[results, accuracy, cv_error] = model_building(model, predictors, output, train_data, test_data)


/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
/home/partha/anaconda/envs/analytics_py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [116]:
results

,Team1,Team2,Actual Winner,Predicted Winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad
1,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,Sunrisers Hyderabad
2,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,Kolkata Knight Riders
3,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,Kings XI Punjab
4,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore
5,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,Sunrisers Hyderabad
6,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,Mumbai Indians
7,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,Royal Challengers Bangalore
8,Delhi Daredevils,Rising Pune Supergiant,Delhi Daredevils,Gujarat Lions
9,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,Mumbai Indians
